In [57]:
import openai
import os
import pandas
import json
import ast
# Replace this with your OpenAI API key
openai.api_key = os.getenv("OPENAI_API_KEY") 

In [63]:
def tag_text_with_llm(play_text):
    prompt = f'''
    You are a tagging assistant for a play script. Given the text, identify:
    - **Speaker** (character speaking)
    - **Stage Directions** (actions, scene instructions)
    - **Utterance** (dialogue lines)

    Please format output as a list of tuples ('speaker', 'instruction', 'utterance'), for empty strings use the list
    delimiter [] and use triple quotes around strings.

    Here’s an example:

    ---
    Text:
    Kerbenne (idet han hidsig kaster Brevet fra sig.) Ak, det havde jeg ikke ventet mig...
    Hortense (nærmer sig sin Fader.) Min kjære Fader, hvad er der i Veien?
    ---
    Output - note use of triple quotes (they can be single quotes or double):
    [
        ("""Kerbenne""", """idet han hidsig kaster Brevet fra sig.""", """Ak, det havde jeg ikke ventet mig..."""),
        ("""Hortense", "nærmer sig sin Fader.", "Min kjære Fader, hvad er der i Veien?"""),
        ...
    ]

    Do not add any text like "Here is the output", "python", or any other explanations or keywords or quotes, just the plain list of tuples
    
    Now process the following text:
    
    {play_text}
    '''
    
    response = openai.chat.completions.create(
        model="gpt-4o-2024-08-06",  
        max_tokens=16000,
        messages=[
            {
                "role": "user",
                "content": prompt,
            },
        ],
    )
    
    return response

In [11]:
# Example usage:
with open("plays/stage_1587758.txt") as fp:
    play_text = fp.read()
    

In [64]:
# Call the function to process
tagged_text = tag_text_with_llm(play_text)

In [33]:
import re

In [65]:
output = tagged_text.choices[0].message.content

In [67]:
output

'```python\n[\n    ("""Kerbenne""", """idet han hidsig kaster Brevet fra sig.""", """Ak, det havde jeg ikke ventet mig"""),\n    ("""Kerbenne""", [] , """nei, det havde jeg ikke"""),\n    ("""Hortense""", """nærmer sig til sin Fader.""", """Min kjære Fader, hvad er der i Veien"""),\n    ("""Kerbenne""", [] , """Denne Proces med min Svoger d\'Ancenis, denne Proces angaaende vort Opgjør, som har været i 10 Aar."""),\n    ("""Hortense""", """smilende:""", """ligesom den troganske Krig"""),\n    ("""Kerbenne""", [] , """Denne Proces, som jeg vandt ved første Instants"""),\n    ("""Hortense""", [] , """Nu vel."""),\n    ("""Kerbenne""", [] , """Nu vel! Ja, nu har jeg tabt den i Paris ved Overretten."""),\n    ("""Kerbenne""", """gaar over paa den venstre Side.""", """Men du fatter vel at jeg ikke lader det bero hermed."""),\n    ("""Kerbenne""", [] , """Jeg er en Bretagner og jeg giver mig ikke. Denne Aften skriver jeg til en ny Advokat"""),\n    ("""Hortense""", [] , """Endnu en. Det blive

In [74]:
play_text[500:1000]

"ol og et Sybord, hvorpaa en Broder. \nramme. Lænestole. Stole \n1ste Scene. \nKertenne \nHortense. \nI det Teppet gaar op sidder Korbenne \nved Skrivebordet tilhøire, han holder \net Brev, som han netop har læst, i \nHaanden. Hortense kommer ind fra \nBaggrunden.) \n\nKerberne. \nidet han hidsig kaster Brevet fra sig.) \nAk, det havde jeg ikke ventet mig \nnei, det havde jeg ikke \nHortense \nnærmer sig til sin Fader.) Min kjære \nFader, hvad er der i Veien \nKerberne. \nDenne Proces med min Svoger d'Ancenis, \nden"

In [76]:
import pandas as pd

In [78]:
df = pd.DataFrame(ast.literal_eval(output.replace("```","").replace("python","")), columns=["Taler", "Scene", "Ytring"])

In [82]:
df

,Taler,Scene,Ytring
0,Kerbenne,idet han hidsig kaster Brevet fra sig.,"Ak, det havde jeg ikke ventet mig"
1,Kerbenne,[],"nei, det havde jeg ikke"
2,Hortense,nærmer sig til sin Fader.,"Min kjære Fader, hvad er der i Veien"
3,Kerbenne,[],"Denne Proces med min Svoger d'Ancenis, denne P..."
4,Hortense,smilende:,ligesom den troganske Krig
...,...,...,...
93,Kerbenne,vred.,Nei! thi jeg er saavel brim.
94,Joceline,[],"På hvem, Herre"
95,Kerbenne,[],"Paa dig, paa mig, paa hele verden. Visitterne,..."
96,Joceline,Går ud i baggrunden til høre.,[]


In [81]:
df.to_excel("stage_1587758.xlsx")

In [43]:
cleaned_output = re.sub(r'[\r\n]+', '', output)  # Remove newlines
#cleaned_output = cleaned_output.replace("'", '"')    # Convert single to double quotes

# Attempt to load the JSON data
try:
    json_data = json.loads(cleaned_output)
except json.JSONDecodeError as e:
    print("JSONDecodeError:", e)
    #print("Raw output for debugging:", output)

JSONDecodeError: Expecting ',' delimiter: line 1 column 43476 (char 43475)


In [49]:
output[43475]

'i'

In [50]:
import re
import json

# Initial cleanup
cleaned_output = re.sub(r'[\r\n]+', '', output)  # Remove newlines
#cleaned_output = cleaned_output.replace("'", '"')  # Convert single to double quotes

# Fix "peaker" typo to "speaker"
cleaned_output = re.sub(r'"peaker":', '"speaker":', cleaned_output)

# Ensure JSON array structure by adding square brackets if missing
if not cleaned_output.strip().startswith("["):
    cleaned_output = "[" + cleaned_output
if not cleaned_output.strip().endswith("]"):
    cleaned_output = cleaned_output + "]"

# Ensure commas between JSON objects
cleaned_output = re.sub(r'}\s*{', '},{', cleaned_output)

# Attempt to load the JSON data
try:
    json_data = json.loads(cleaned_output)
except json.JSONDecodeError as e:
    print("JSONDecodeError:", e)
    print("Cleaned output for debugging:", cleaned_output)
    json_data = None


JSONDecodeError: Expecting ',' delimiter: line 1 column 43630 (char 43629)
Cleaned output for debugging: [    {        "speaker": "Kerbenne",        "instruction": "idet han hidsig kaster Brevet fra sig.",        "utterance": "Ak, det havde jeg ikke ventet mig nei, det havde jeg ikke"    },    {        "speaker": "Hortense",        "instruction": "nærmer sig til sin Fader.",        "utterance": "Min kjære Fader, hvad er der i Veien"    },    {        "speaker": "Kerbenne",        "utterance": "Denne Proces med min Svoger d'Ancenis, denne Proces angaaende vort Opgjør, som har været i 10 Aar."    },    {        "speaker": "Hortense",        "instruction": "smilende:",        "utterance": "ligesom den troganske Krig."    },    {        "speaker": "Kerbenne",        "utterance": "Denne Proces, som jeg vandt ved første Instants"    },    {        "speaker": "Hortense",        "utterance": "Nu vel."    },    {        "speaker": "Kerbenne",        "instruction": "gaar over paa den venstre Sid